In [10]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v3 as iio
from moviepy.video.io.bindings import mplfig_to_npimage
from io import BytesIO

In [11]:
def calculate_best_point_luke_min_max(xd, yd):
    x = -np.cos(np.arctan2(yd, xd))
    y = -np.sin(np.arctan2(yd, xd))
    return x, y

def calculate_best_point_luke_escape(xl, yl):
    x = np.cos(np.arctan2(yl, xl))
    y = np.sin(np.arctan2(yl, xl))
    return x, y

def calculate_best_point_darth(xl, yl):
    x = np.cos(np.arctan2(yl, xl))
    y = np.sin(np.arctan2(yl, xl))
    return x, y

def update_position_luke(x0, y0, x1, y1, time_step):
    theta = np.arctan2(y1 - y0, x1 - x0)
    x = x0 + time_step * np.cos(theta)
    y = y0 + time_step * np.sin(theta)
    return x, y

def update_position_darth(x0, y0, k, direction, time_step):
    theta_0 = np.arctan2(y0, x0)
    # Moves clockwise or counterclockwise on a circunference with speed 1 based on the best point to go
    x = np.cos(direction*k*time_step + theta_0)
    y = np.sin(direction*k*time_step + theta_0)
    return x, y

def check_best_path(xl0, yl0, xl1, yl1):
    if xl0 * yl1 - yl0 * xl1 > 0:
        return 1
    else:
        return -1

    
def check_end(xl, yl, xd, yd, tol = 1e-4):
    rl = np.sqrt(xl**2 + yl**2)
    distance = np.sqrt((xl - xd)**2 + (yl - yd)**2)

    if distance < tol:
        print("Darth got Luke!")
        print("Final position (Darth): ", xd, yd)
        print("Final position (Luke): ", xl, yl)
        print("Final distance: ", distance)
        return True, "Darth"
    elif rl >= 1 - tol:
        print("Luke Escaped!")
        print("Final position (Darth): ", xd, yd)
        print("Final position (Luke): ", xl, yl)
        print("Final distance: ", distance)
        return True, "Luke"
    else:
        return False, "None"

def distance(xl, yl, xd, yd):
    return np.sqrt((xl - xd)**2 + (yl - yd)**2)

def reach_point_darth(xd0, yd0, xd1, yd1, k):
    # Returns in how many time steps Darth will reach the point (xd1, yd1)
    theta_0 = np.arctan2(xd0, yd0)
    if theta_0 < 0:
        theta_0 += 2*np.pi
    if theta_0 >= 2*np.pi - 1e-6:
        theta_0 %= 2*np.pi

    theta_1 = np.arctan2(xd1, yd1)
    if theta_1 < 0:
        theta_1 += 2*np.pi
    if theta_1 >= 2*np.pi - 1e-6:
        theta_1 %= 2*np.pi

    angular_speed = k
    time = np.abs(theta_1 - theta_0)/angular_speed
    return time

def reach_point_luke(xl0, yl0, xl1, yl1):
    # Returns in how many time steps Luke will reach the point (xl1, yl1)
    distance = np.sqrt((xl1 - xl0)**2 + (yl1 - yl0)**2)
    return distance 

def check_if_will_be_intercepted(xl0, yl0, xd0, yd0, xl1, yl1, k):
    if reach_point_luke(xl0, yl0, xl1, yl1) >= reach_point_darth(xd0, yd0, xl1, yl1, k):
        return True
    else:
        return False

def save_frame(xl, yl, xd, yd, xl1, yl1, xd1, yd1, time_estimated_luke, time_estimated_darth, i):
    fig = plt.figure()
    #plot circle

    theta = np.linspace(0, 2*np.pi, 100)
    x = np.cos(theta)
    y = np.sin(theta)
    plt.plot(x, y)

    # Plot the trajectory
    plt.plot(xl, yl, 'b')
    plt.plot(xd, yd, 'r')

    # Plot the final position
    plt.plot(xl[-1], yl[-1], 'bo')
    plt.plot(xd[-1], yd[-1], 'ro')

    # Plot the intended position
    plt.plot(xl1, yl1, 'bx', markersize = 5)
    plt.plot(xd1, yd1, 'rx', markersize = 5)

    # Plot the estimated time to reach the point
    plt.title("Estimated time to reach the point (Luke): " + str(time_estimated_luke) + 
              "\n Estimated time to reach the point (Darth): " + str(time_estimated_darth))

    plt.axis('equal')
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    plt.close(fig)
    return iio.imread(buffer.getvalue())

def simulate(xl0 = 1e-6, yl0 = 0.5, xd0 = 0, yd0 = 1, k = 0.01, t_end = 5, step = 1000, fps = 60):
    xl = []
    yl = []
    xd = []
    yd = []

    xl.append(xl0)
    yl.append(yl0)
    xd.append(xd0)
    yd.append(yd0)

    frames = list()

    for i in range(step):
        xd1, yd1 = calculate_best_point_darth(xl0, yl0)
        xlminimum, ylminimum = calculate_best_point_luke_escape(xl0, yl0)

        time_estimated_luke = reach_point_luke(xl0, yl0, xlminimum, ylminimum)
        time_estimated_darth = reach_point_darth(xd0, yd0, xlminimum, ylminimum, k)

        if check_if_will_be_intercepted(xl0, yl0, xd0, yd0, xlminimum, ylminimum, k):
            xl1, yl1 = calculate_best_point_luke_min_max(xd0, yd0)
        else:
            xl1, yl1 = calculate_best_point_luke_escape(xl0, yl0)
        direction = check_best_path(xd0, yd0, xd1, yd1)

        xl0, yl0 = update_position_luke(xl0, yl0, xl1, yl1, t_end/step)
        xd0, yd0 = update_position_darth(xd0, yd0, k, direction, t_end/step)

        xl.append(xl0)
        yl.append(yl0)
        xd.append(xd0)
        yd.append(yd0)

        if i % ((step // t_end) // fps)  == 0:
            frames.append(save_frame(xl, yl, xd, yd, xl1, yl1, xd1, yd1, 
                                     time_estimated_luke, time_estimated_darth, i))

        end, winner = check_end(xl0, yl0, xd0, yd0)
        if end:
            frames.append(save_frame(xl, yl, xd, yd, xl1, yl1, xd1, yd1, 
                                     time_estimated_luke, time_estimated_darth, i))
            t_end = t_end*i/step
            break

    iio.imwrite('simulation.mp4', frames, fps = fps)
    print("Simulation ended at time step ", i+1, " of ", step)
    print("Winner is: ", winner)
    print("Time elapsed: ", t_end, " seconds")
    print("***************************************************")

In [12]:
time = 5
step = int(1e6)

simulate(t_end = time, step = step, k = 1.5*np.pi)

Simulation ended at time step  1000000  of  1000000
Winner is:  None
Time elapsed:  5  seconds
***************************************************
